In [ ]:
import os
import sys

"""Linux"""
os.chdir("/home/optimusprime/Desktop/peeterson/github/DeepAR_demand_prediction/Rental_Fleet_Demand_Forecast")
sys.path.append(os.path.abspath(os.path.join("/home/optimusprime/Desktop/peeterson/github/DeepAR_demand_prediction/Rental_Fleet_Demand_Forecast")))

os.chdir("/home/optimusprime/Desktop/peeterson/github/DeepAR_demand_prediction/Rental_Fleet_Demand_Forecast/libs/linux-gpu")


#from ctypes import FormatError
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import os,sys

import torch
torch.use_deterministic_algorithms(True)
import pytorch_lightning as pl
from pytorch_forecasting.data.encoders import TorchNormalizer
from pytorch_forecasting.metrics import SMAPE, RMSE
from torchmetrics import R2Score, SymmetricMeanAbsolutePercentageError, MeanSquaredError

import matplotlib.pyplot as plt
import pandas as pd
from pytorch_forecasting.data import TimeSeriesDataSet
from pytorch_forecasting.data.encoders import NaNLabelEncoder
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
import torch
from pytorch_forecasting.data.encoders import TorchNormalizer
import os,sys
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import acf,pacf
from scipy.signal import find_peaks
import operator
import statsmodels.api as sm
from itertools import combinations
import pickle
from pytorch_forecasting import Baseline
import random
from pytorch_forecasting import DeepAR,NegativeBinomialDistributionLoss
from itertools import product
from sklearn.metrics import mean_absolute_error, mean_squared_error
import optuna
from optuna.trial import TrialState
import plotly
#from deepar_RegionWise_LinuxGpu_prediction_dev import train_and_forecast
from sklearn.metrics import classification_report

"""
Set Random seed
"""

random.seed(0)
torch.manual_seed(0)
np.random.seed(0)
## additional seeding to ensure reproduciblility.
pl.seed_everything(0)

2024-04-02 16:22:49.376699: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-02 16:22:49.377926: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-02 16:22:49.396271: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 16:22:49.396292: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 16:22:49.396807: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

0

Now for the round 0, we reduce dataset even more from 20% to 10%. num-epochs to a fixed value of 50. number of trials to a fixed number of 50.

In [ ]:
dataset_size = 0.05
num_epochs = 30
num_trials = 30
logs_directory = "/home/optimusprime/Desktop/Timothy_EXP_5/Round_1_logs"

In [ ]:
"""
Import pre-processed Data (tsfresh features)

response and target are the same thing
"""

### Your code
os.chdir('/home/optimusprime/Desktop/Data_Timothy')
############ Basically df = raw_feat_df <-------------------------
df = pd.read_parquet('EXP_5_combined_dd_ss_raw_feat_data_cluster_aggregation.parquet', engine='pyarrow')
df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,"inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_0","inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_1",inflow_diff2_per1__kurtosis,inflow_diff2_per1__autocorrelation__lag_2,inflow_diff2_per1__autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_3,inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__skewness,inflow_diff2_per1__count_above_mean
0,167,2021-01-07 23:00:00,50,59.0,107.0,38.0,12.0,117.0,122.0,41.0,...,-3.0,-4.786351,0.561537,-0.128245,-0.269178,-0.269166,-0.110965,-0.026810,-0.468564,0.0
1,168,2021-01-08 00:00:00,50,32.0,67.0,14.0,30.0,158.0,59.0,23.0,...,4.0,-10.000000,0.561537,-0.128245,-1.000000,-0.269166,-0.110965,-0.026810,-0.468564,1.0
2,169,2021-01-08 01:00:00,50,32.0,34.0,17.0,84.0,120.0,32.0,23.0,...,27.0,-18.000000,0.561537,-1.465116,-0.017442,-0.269166,-0.110965,-0.026810,0.670284,1.0
3,170,2021-01-08 02:00:00,50,24.0,24.0,4.0,162.0,63.0,32.0,8.0,...,25.0,-26.000000,0.891341,-0.741231,-0.406555,-0.406555,-0.110965,0.702128,1.279001,2.0
4,171,2021-01-08 03:00:00,50,9.0,12.0,12.0,243.0,24.0,24.0,27.0,...,31.0,-15.972136,1.663525,-0.630481,-0.373850,-0.373850,-0.110965,0.432843,1.254480,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247120,24808,2023-10-31 16:00:00,246,937.0,833.0,866.0,701.0,161.0,1060.0,1083.0,...,18.0,-155.248275,0.388287,-0.356099,0.135241,0.135241,0.053212,-0.073461,-0.347820,12.0
247121,24809,2023-10-31 17:00:00,246,884.0,797.0,842.0,560.0,75.0,937.0,979.0,...,-7.0,-110.884152,0.401069,-0.358260,0.141407,0.141407,0.077524,-0.060138,-0.317217,12.0
247122,24810,2023-10-31 18:00:00,246,843.0,766.0,772.0,324.0,37.0,884.0,959.0,...,68.0,38.090968,0.503437,-0.384094,0.142216,0.142216,0.048863,-0.096132,-0.416259,13.0
247123,24811,2023-10-31 19:00:00,246,735.0,764.0,701.0,161.0,32.0,843.0,791.0,...,-20.0,24.437373,0.209153,-0.370867,0.126136,0.126136,0.053884,-0.069905,-0.314286,13.0


In [ ]:
working_directory = '/home/optimusprime/Desktop/Timothy_EXP_5'
os.chdir(working_directory)

In [ ]:
unique_values_group_no_ts_fresh = list(df['group'].unique())
print(len(unique_values_group_no_ts_fresh), "\n")
print(unique_values_group_no_ts_fresh)

10 

[50, 74, 120, 125, 141, 151, 232, 238, 239, 246]


In [ ]:
zones_time_idx_no_ts_fresh = {}
for i in unique_values_group_no_ts_fresh:
    df1 = df[df['group'] == i]
    time_idx_list = df1['time_idx'].tolist()
    first_idx = time_idx_list[0]
    last_idx = time_idx_list[-1]
    zones_time_idx_no_ts_fresh[i] = [first_idx, last_idx]
    print("Group ", i, " has ", df1.shape[0], " number of rows in df_ts_fresh \n")
    print("First idx: ", first_idx, "; Last idx: ", last_idx, "\n")

Group  50  has  24646  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  24812 

Group  74  has  24645  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  24812 

Group  120  has  24715  number of rows in df_ts_fresh 

First idx:  98 ; Last idx:  24812 

Group  125  has  24670  number of rows in df_ts_fresh 

First idx:  143 ; Last idx:  24812 

Group  141  has  24790  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  24812 

Group  151  has  24645  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  24812 

Group  232  has  24646  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  24812 

Group  238  has  24790  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  24812 

Group  239  has  24790  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  24812 

Group  246  has  24788  number of rows in df_ts_fresh 

First idx:  25 ; Last idx:  24812 



In [ ]:
first_idx = df.loc[df['group'] == 141, 'time_idx'].iloc[0]
total_time_idx = df[df.group == 141].shape[0]
reduced_df_time_idx = [*range(first_idx, int(dataset_size*total_time_idx)+1, 1)]
df = df[df['time_idx'].isin(reduced_df_time_idx)]

In [ ]:
numeric_cols = list(df.columns)
numeric_cols = [e for e in numeric_cols if e not in ('time_idx', 'datetime', 'group', 'target')]
print(numeric_cols)

['inflow_lag_0', 'inflow_lag_1', 'inflow_lag_2', 'inflow_lag_3', 'outflow_lag_0', 'outflow_lag_1', 'outflow_lag_2', 'outflow_lag_3', 'inflow', 'inflow_diff1_per1', 'inflow_diff1_per2', 'inflow_diff2_per1', 'inflow_lag_0__fft_coefficient__attr_"real"__coeff_0', 'inflow_lag_0__fft_coefficient__attr_"real"__coeff_1', 'inflow_lag_0__skewness', 'inflow_lag_0__count_above_mean', 'inflow_lag_0__autocorrelation__lag_1', 'inflow_lag_0__count_below_mean', 'inflow_lag_0__autocorrelation__lag_2', 'inflow_lag_0__autocorrelation__lag_3', 'inflow_lag_0__partial_autocorrelation__lag_2', 'inflow_lag_0__partial_autocorrelation__lag_1', 'inflow_lag_0__partial_autocorrelation__lag_3', 'inflow_lag_0__kurtosis', 'inflow_lag_1__fft_coefficient__attr_"real"__coeff_0', 'inflow_lag_1__fft_coefficient__attr_"real"__coeff_1', 'inflow_lag_1__skewness', 'inflow_lag_1__count_above_mean', 'inflow_lag_1__autocorrelation__lag_1', 'inflow_lag_1__count_below_mean', 'inflow_lag_1__autocorrelation__lag_2', 'inflow_lag_1__a

In [ ]:
#################### add date information ts ####################

# create datetime features from datetime column in raw_feat_df
df['_hour_of_day'] = df["datetime"].dt.hour.astype(str)
df['_day_of_week'] = df["datetime"].dt.dayofweek.astype(str)
df['_day_of_month'] = df["datetime"].dt.day.astype(str)
df['_day_of_year'] = df["datetime"].dt.dayofyear.astype(str)
df['_week_of_year'] = df["datetime"].dt.weekofyear.astype(str)
df['_month_of_year'] = df["datetime"].dt.month.astype(str)
df['_year'] = df["datetime"].dt.year.astype(str)
#################### add date information ts ####################

hour_of_day_nunique = df['_hour_of_day'].nunique()
day_of_week_nunique = df['_day_of_week'].nunique()
day_of_month_nunique = df['_day_of_month'].nunique()
day_of_year_nunique = df['_day_of_year'].nunique()
week_of_year_nunique = df['_week_of_year'].nunique()
month_of_year_nunique = df['_month_of_year'].nunique()
year_nunique = df['_year'].nunique()

In [ ]:
df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__skewness,inflow_diff2_per1__count_above_mean,_hour_of_day,_day_of_week,_day_of_month,_day_of_year,_week_of_year,_month_of_year,_year
0,167,2021-01-07 23:00:00,50,59.0,107.0,38.0,12.0,117.0,122.0,41.0,...,-0.026810,-0.468564,0.0,23,3,7,7,1,1,2021
1,168,2021-01-08 00:00:00,50,32.0,67.0,14.0,30.0,158.0,59.0,23.0,...,-0.026810,-0.468564,1.0,0,4,8,8,1,1,2021
2,169,2021-01-08 01:00:00,50,32.0,34.0,17.0,84.0,120.0,32.0,23.0,...,-0.026810,0.670284,1.0,1,4,8,8,1,1,2021
3,170,2021-01-08 02:00:00,50,24.0,24.0,4.0,162.0,63.0,32.0,8.0,...,0.702128,1.279001,2.0,2,4,8,8,1,1,2021
4,171,2021-01-08 03:00:00,50,9.0,12.0,12.0,243.0,24.0,24.0,27.0,...,0.432843,1.254480,2.0,3,4,8,8,1,1,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223547,1235,2021-02-21 11:00:00,246,283.0,210.0,305.0,453.0,495.0,256.0,348.0,...,-0.041864,-0.441817,14.0,11,6,21,52,7,2,2021
223548,1236,2021-02-21 12:00:00,246,404.0,270.0,420.0,484.0,481.0,283.0,410.0,...,-0.038264,-0.398056,15.0,12,6,21,52,7,2,2021
223549,1237,2021-02-21 13:00:00,246,407.0,331.0,411.0,478.0,504.0,404.0,499.0,...,-0.149345,-0.191969,13.0,13,6,21,52,7,2,2021
223550,1238,2021-02-21 14:00:00,246,395.0,380.0,453.0,495.0,412.0,407.0,516.0,...,-0.168011,-0.059596,13.0,14,6,21,52,7,2,2021


In [ ]:
print(hour_of_day_nunique)
print(day_of_week_nunique)
print(day_of_month_nunique)
print(day_of_year_nunique)
print(week_of_year_nunique)
print(month_of_year_nunique)
print(year_nunique)

24
7
31
52
8
2
1


In [ ]:
unique_values_group_no_ts_fresh = list(df['group'].unique())
print(len(unique_values_group_no_ts_fresh), "\n")
print(unique_values_group_no_ts_fresh)

10 

[50, 74, 120, 125, 141, 151, 232, 238, 239, 246]


In [ ]:
zones_time_idx_no_ts_fresh = {}
for i in unique_values_group_no_ts_fresh:
    df1 = df[df['group'] == i]
    time_idx_list = df1['time_idx'].tolist()
    first_idx = time_idx_list[0]
    last_idx = time_idx_list[-1]
    zones_time_idx_no_ts_fresh[i] = [first_idx, last_idx]
    print("Group ", i, " has ", df1.shape[0], " number of rows in df_ts_fresh \n")
    print("First idx: ", first_idx, "; Last idx: ", last_idx, "\n")

Group  50  has  1073  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  1239 

Group  74  has  1072  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  1239 

Group  120  has  1142  number of rows in df_ts_fresh 

First idx:  98 ; Last idx:  1239 

Group  125  has  1097  number of rows in df_ts_fresh 

First idx:  143 ; Last idx:  1239 

Group  141  has  1217  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  1239 

Group  151  has  1072  number of rows in df_ts_fresh 

First idx:  168 ; Last idx:  1239 

Group  232  has  1073  number of rows in df_ts_fresh 

First idx:  167 ; Last idx:  1239 

Group  238  has  1217  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  1239 

Group  239  has  1217  number of rows in df_ts_fresh 

First idx:  23 ; Last idx:  1239 

Group  246  has  1215  number of rows in df_ts_fresh 

First idx:  25 ; Last idx:  1239 



In [ ]:
encoder_len = 8 # looks back 8 hours in the past for prediction
pred_len = 1 # 1, 6, 12 # how many timeseteps you want to predict in the future (forecasting), 1 means present (nowcasting)
#cov_lag_len= pred_len

# not using covariates yet, so put as 0
cov_lag_len= 0 #we can use forecasted values, even for inflow

####### train val test split of tsfresh dataframe #########

### you want a the idx for each train, validation and test

### train = 0.7 of each zone (eg. time idx 167 to roughly 20000) eg train = 0.7 * (24812-167)
### validation = 0.2 of each zone (eg. time idx 20000 to 22000)
### test = 0.1 of each zone (eg. time idx 22000 to 24000)

Target = 'target'

total_time_idx = df[df.group == 141].shape[0]

first_idx = df.loc[df['group'] == 141, 'time_idx'].iloc[0]

tr_stop_idx = int(0.7*total_time_idx)
val_start_idx = tr_stop_idx - encoder_len
val_stop_idx = total_time_idx-1
# val_stop_idx = val_start_idx + int(0.2*total_time_idx) # use (tr_stop_idx - encoder_len) as encoder from train set for validatoin
# tes_start_idx = val_stop_idx - encoder_len
# tes_stop_idx = total_time_idx-1

print(tr_stop_idx)
print(val_start_idx)
print(val_stop_idx)
# print(tes_start_idx)
# print(tes_stop_idx)

851
843
1216


In [ ]:
train_time_idx = [*range(first_idx, first_idx+tr_stop_idx+1, 1)]
print(train_time_idx[0], train_time_idx[-1])
val_time_idx = [*range(first_idx+val_start_idx, first_idx+val_stop_idx+1, 1)]
print(val_time_idx[0], val_time_idx[-1])
# test_time_idx = [*range(first_idx+tes_start_idx, first_idx+tes_stop_idx+1, 1)]
# print(test_time_idx[0], test_time_idx[-1])

23 874
866 1239


In [ ]:
# unique_zones = list(df['group'].unique())

In [ ]:
"""
set inputs here
(hyperparameters grid search)

"""
######### Network Architecture ###################
# p = 10 # patience no. of epochs

Loss=NegativeBinomialDistributionLoss() # because we assume that the demand follows a negative bionmial distribution

######### Network Architecture ###################


######### Training Routine ###################
fdv_steps = 10 # fast_dev_run -> small set of training to see if training is happening correctly (one of the params in deepar model)
######### Training Routine ###################


# this is for naive forecast, getting past week's demand and predicting purely based on that
############## Inputs for 2) Persistance model ( seasonal naive forecast ) #######################
season_len = 168 # length of season ()
num_past_seas = 6 # number of past seasons to use in averaging (look at 6 week's back, getting average of these 6 values)
#seas_pred_strt_idx = 2035 # seasonal naive forecast start index, in hours use the df dataframe
############## Inputs for 2) Persistance model ( seasonal naive forecast ) #######################


In [ ]:
"""
Full Training Routine
with hyperparmeter grid search

Load data into TimeSeriesDataSet object

for fast development run
uncomment fast_dev_run = fdv_steps

"""

# early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=p, verbose=False, mode="min")
lr_logger = LearningRateMonitor()

# param_comb_cnt=-1
#for neu,lay,bat,lr,enc_len,pred_len,drop,cov_pair,num_ep in product(*[x for x in hparams_grid.values()]):


file_path = "trial_parameters.json"
# Check if the file exists before attempting to delete it
if os.path.exists(file_path):
    # Delete the file
    os.remove(file_path)

import json
def save_trial_parameters(study, trial):
    trial_params = {'trial_num':trial.number ,'trial_params' :trial.params}
    with open("trial_parameters.json", "a") as f:
        json.dump(trial_params, f)
        f.write('\n')



def objective(trial): #optuna objective function to find best hyperparameters
# minimize the average validation loss

  # optuna tries to find best combination given this hyperparameter searchspace

  # first start broad
  neu = trial.suggest_int(name="neu",low=10,high=1000,step=100,log=False) # number of neurons in lstm cell
  lay = trial.suggest_int(name="lay",low=1,high=10,step=2,log=False) # number of lstm layers
  bat = trial.suggest_int(name="bat",low=32,high=1024,step=32,log=False) # batch size
  lr = trial.suggest_float(name="lr",low=0.00001,high=0.1,log=True) #learning rate, log does not have step
  drop = trial.suggest_float(name="dropout",low=0,high=0.8,step=0.2,log=False) # dont use some of the cells to distribute learning across the cells

  num_cols_list = numeric_cols
  cat_dict = {"_hour_of_day": NaNLabelEncoder(add_nan=True).fit(df._hour_of_day),
          "_day_of_week": NaNLabelEncoder(add_nan=True).fit(df._day_of_week), "_day_of_month" : NaNLabelEncoder(add_nan=True).fit(df._day_of_month), "_day_of_year" : NaNLabelEncoder(add_nan=True).fit(df._day_of_year),
              "_week_of_year": NaNLabelEncoder(add_nan=True).fit(df._week_of_year), "_month_of_year": NaNLabelEncoder(add_nan=True).fit(df._month_of_year) ,"_year": NaNLabelEncoder(add_nan=True).fit(df._year)}
  cat_list = ["_hour_of_day","_day_of_week","_day_of_month","_day_of_year","_week_of_year","_month_of_year","_year"]
  embed_size_dict = {'_hour_of_day':(hour_of_day_nunique+1,5),'_day_of_week':(day_of_week_nunique+1,5),'_day_of_month':(day_of_month_nunique+1,5), '_day_of_year':(day_of_year_nunique+1,8), '_week_of_year':(week_of_year_nunique+1,3), '_month_of_year':(month_of_year_nunique+1,3),'_year':(year_nunique+1,1)}
  # ^ converting hour into a 5 dimensional vector, etc etc, day of year = number of unique values in this column + 1, same for week of year, month of year and year

  train_dataset = TimeSeriesDataSet(
      df[(df['time_idx'].isin(train_time_idx))],
      time_idx="time_idx",
      target=Target,
      categorical_encoders=cat_dict,
      group_ids=["group"],
      min_encoder_length=encoder_len,
      max_encoder_length=encoder_len,
      min_prediction_length=pred_len,
      max_prediction_length=pred_len,
      time_varying_unknown_reals=[Target],
      time_varying_known_reals=num_cols_list,
      time_varying_known_categoricals=cat_list,
      add_relative_time_idx=False,
      randomize_length=False,
      scalers={},
      target_normalizer=TorchNormalizer(method="identity",center=False,transformation=None)
    )

  val_dataset = TimeSeriesDataSet.from_dataset(train_dataset,df[(df['time_idx'].isin(val_time_idx))], stop_randomization=True, predict=False)
  # test_dataset = TimeSeriesDataSet.from_dataset(train_dataset,df[df['time_idx'].isin(test_time_idx)], stop_randomization=True)

  train_dataloader = train_dataset.to_dataloader(train=True, batch_size=bat)
  val_dataloader = val_dataset.to_dataloader(train=False, batch_size=bat)
  # test_dataloader = test_dataset.to_dataloader(train=False, batch_size=bat)


  # trial.set_user_attr("num_ep", num_ep)

  early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=5,
    verbose=True,
    mode='min'
  )


  """
  Machine Learning predictions START
  1) DeepAR

  """
  trainer = pl.Trainer(
      max_epochs=num_epochs,
      accelerator='gpu',
      devices=[0],
      gradient_clip_val=0.1,
      limit_train_batches=1.0,
      limit_val_batches=1.0,
      # fast_dev_run=fdv_steps,
      logger=True,
      callbacks=[lr_logger, early_stop_callback],    #, early_stop_callback]
      default_root_dir=logs_directory
  )


  #print(f"training routing:\n \n {trainer}")
  deepar = DeepAR.from_dataset(
      train_dataset,
      learning_rate=lr,
      hidden_size=neu,
      rnn_layers=lay,
      dropout=drop,
      loss=Loss,
      log_interval=20,
      log_val_interval=6,
      log_gradient_flow=False,
      embedding_sizes=embed_size_dict
      # reduce_on_plateau_patience=3,
  )

  #print(f"Number of parameters in network: {deepar.size()/1e3:.1f}k")
  # print(f"Model :\n \n {deepar}")
  torch.set_num_threads(10)
  trainer.fit(
      deepar,
      train_dataloaders=train_dataloader,
      val_dataloaders=val_dataloader,
  )

  val_wmape = trainer.callback_metrics['val_WMAPE'].item()
  # val_wmape = trainer.callback_metrics['val_WMAPE'].item()
  trial.report(val_wmape, num_epochs)
  # trial.report(val_wmape, num_ep)

  # Handle pruning based on the intermediate value.
  if trial.should_prune():
      raise optuna.exceptions.TrialPruned()

  return val_wmape #optuna is trying to minimise this <---


######### optuna results #####################
if __name__ == "__main__":

  study = optuna.create_study(direction="minimize")
  study.optimize(objective, n_trials=num_trials,callbacks=[save_trial_parameters]) # can specify the number of trials to a number less than 960 for example, eg can set n_trials = 100

  pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
  complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

  print("Study statistics: ")
  print("  Number of finished trials: ", len(study.trials))
  print("  Number of pruned trials: ", len(pruned_trials))
  print("  Number of complete trials: ", len(complete_trials))

  print("Best trial:")
  trial = study.best_trial
  # best_num_ep = study.trials[trial.number].user_attrs['num_ep']

  print("  Value: ", trial.value)

  print("  Params: ")
  for key, value in trial.params.items():
      print("    {}: {}".format(key, value))
########## optuna results #####################

[I 2024-04-02 16:22:51,635] A new study created in memory with name: no-name-afde5687-b520-4f1e-9893-8f35aeabb952
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
Missing logger folder: /home/optimusprime/Desktop/Timothy_EXP_5/Round_1_logs/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                             | Params
----------------------------------------------------------------------------
0 | loss                   | NegativeBinomialDistributionLoss | 0     
1 | logging_metrics        | ModuleList                       | 0     
2 | embeddings             | MultiEmbedding                   | 787   
3 | rnn                    | LSTM                     

Epoch 0: 100%|██████████| 39/39 [00:04<00:00,  8.90it/s, loss=33.1, v_num=0, train_loss_step=7.600, val_loss=7.570]

Metric val_loss improved. New best score: 7.565


Epoch 1: 100%|██████████| 39/39 [00:04<00:00,  9.02it/s, loss=6.74, v_num=0, train_loss_step=6.450, val_loss=6.550, train_loss_epoch=72.20]

Metric val_loss improved by 1.014 >= min_delta = 0. New best score: 6.551


...

Epoch 29: 100%|██████████| 39/39 [00:04<00:00,  9.17it/s, loss=4.43, v_num=29, train_loss_step=4.610, val_loss=4.720, train_loss_epoch=4.430]

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|██████████| 39/39 [00:04<00:00,  8.62it/s, loss=4.43, v_num=29, train_loss_step=4.610, val_loss=4.720, train_loss_epoch=4.430]

[I 2024-04-02 17:35:20,060] Trial 29 finished with value: 15.439125061035156 and parameters: {'neu': 710, 'lay': 3, 'bat': 288, 'lr': 0.02185049073942728, 'dropout': 0.0}. Best is trial 23 with value: 11.80363941192627.



Study statistics: 
  Number of finished trials:  30
  Number of pruned trials:  9
  Number of complete trials:  21
Best trial:
  Value:  11.80363941192627
  Params: 
    neu: 210
    lay: 3
    bat: 32
    lr: 0.01644130325721595
    dropout: 0.6000000000000001


In [ ]:
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("  Best trial: ")
print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# print("Best number of epochs: ", best_num_ep)

Study statistics: 
  Number of finished trials:  30
  Number of pruned trials:  9
  Number of complete trials:  21
  Best trial: 
  Value:  11.80363941192627
  Params: 
    neu: 210
    lay: 3
    bat: 32
    lr: 0.01644130325721595
    dropout: 0.6000000000000001


In [ ]:
from IPython.display import display
fig = optuna.visualization.plot_parallel_coordinate(study)
display(fig)
fig = optuna.visualization.plot_optimization_history(study)
display(fig)
fig = optuna.visualization.plot_slice(study)
display(fig)
fig = optuna.visualization.plot_param_importances(study)
display(fig)

In [ ]:
print("  Best trial: ", trial)

  Best trial:  FrozenTrial(number=23, state=TrialState.COMPLETE, values=[11.80363941192627], datetime_start=datetime.datetime(2024, 4, 2, 17, 18, 51, 95792), datetime_complete=datetime.datetime(2024, 4, 2, 17, 22, 46, 645394), params={'neu': 210, 'lay': 3, 'bat': 32, 'lr': 0.01644130325721595, 'dropout': 0.6000000000000001}, user_attrs={}, system_attrs={}, intermediate_values={30: 11.80363941192627}, distributions={'neu': IntDistribution(high=910, log=False, low=10, step=100), 'lay': IntDistribution(high=9, log=False, low=1, step=2), 'bat': IntDistribution(high=1024, log=False, low=32, step=32), 'lr': FloatDistribution(high=0.1, log=True, low=1e-05, step=None), 'dropout': FloatDistribution(high=0.8, log=False, low=0.0, step=0.2)}, trial_id=23, value=None)
